# Travelling Salesperson Problem solved using genetic algorithms

In [1]:
# Imports 
import numpy as np
import random

from datetime import datetime

In [2]:
# Parameters
n_cities = 20

n_population = 100

mutation_rate = 0.3

In [3]:
# Generating a list of coordenades representing each city
coordinates_list = [[x,y] for x,y in zip(np.random.randint(0,100,n_cities),np.random.randint(0,100,n_cities))]
names_list = np.array(['Berlin', 'London', 'Moscow', 'Barcelona', 'Rome', 'Paris', 'Vienna', 'Munich', 'Istanbul', 'Kyiv', 'Bucharest', 'Minsk', 'Warsaw', 'Budapest', 'Milan', 'Prague', 'Sofia', 'Birmingham', 'Brussels', 'Amsterdam'])
cities_dict = { x:y for x,y in zip(names_list,coordinates_list)}

# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

cities_dict

{'Berlin': [29, 72],
 'London': [54, 65],
 'Moscow': [72, 21],
 'Barcelona': [38, 50],
 'Rome': [20, 60],
 'Paris': [48, 31],
 'Vienna': [54, 57],
 'Munich': [39, 5],
 'Istanbul': [50, 46],
 'Kyiv': [95, 58],
 'Bucharest': [80, 74],
 'Minsk': [56, 34],
 'Warsaw': [94, 39],
 'Budapest': [75, 22],
 'Milan': [58, 39],
 'Prague': [35, 49],
 'Sofia': [69, 28],
 'Birmingham': [6, 91],
 'Brussels': [52, 63],
 'Amsterdam': [67, 94]}

## 1. Create the first population set
We randomly shuffle the cities N times where N=population_size

In [4]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
population_set

array([['Vienna', 'Warsaw', 'Sofia', ..., 'Bucharest', 'Paris',
        'Brussels'],
       ['Rome', 'Minsk', 'Bucharest', ..., 'London', 'Milan', 'Vienna'],
       ['London', 'Milan', 'Vienna', ..., 'Rome', 'Sofia', 'Berlin'],
       ...,
       ['Munich', 'Amsterdam', 'Budapest', ..., 'Vienna', 'Barcelona',
        'Sofia'],
       ['Barcelona', 'Paris', 'Minsk', ..., 'Rome', 'Birmingham',
        'Istanbul'],
       ['Bucharest', 'Kyiv', 'Barcelona', ..., 'Amsterdam', 'Sofia',
        'Munich']], dtype='<U10')

## 2. Evaluate solutions fitness
The solutions are defined so that the first element on the list is the first city to visit, then the second, etc. and the last city is linked to the first.
The fitness function needs to compute the distance between subsequent cities.

In [5]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

In [6]:
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([871.54746094, 783.466072  , 818.80547691, 814.20084771,
       843.08481584, 826.30429897, 755.33598176, 753.41998128,
       699.19500777, 781.11362743, 810.24700885, 657.65663928,
       822.91027133, 813.95802082, 760.33966268, 838.60007828,
       749.15471071, 776.12562267, 850.16054821, 758.41601341,
       787.94972642, 823.38360732, 841.23670576, 763.3748384 ,
       843.48112396, 751.02276219, 840.6953748 , 754.10349637,
       679.82803282, 872.66301195, 837.00451015, 790.53706032,
       827.07775239, 665.3926298 , 852.51135667, 755.559591  ,
       808.17886608, 813.13532984, 788.55354178, 825.37873394,
       820.26228573, 762.74323329, 682.83946865, 705.40199871,
       763.68171239, 679.05066857, 782.27108768, 665.97640553,
       835.82274867, 799.91894701, 866.19917173, 770.93231326,
       760.9064194 , 590.74752431, 745.66313186, 726.73446452,
       895.6383672 , 796.90892094, 862.27339967, 845.78297077,
       754.70262037, 687.52247596, 732.36380761, 707.12

# 3. Progenitors selection
I will select a new set of progenitors using the Roulette Wheel Selection. Generates a list of progenitor pairs where N= len(population_set) but at each position there are two solutions to merge

In [7]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
progenitor_list[0][2]

array(['Vienna', 'Minsk', 'Kyiv', 'Amsterdam', 'Barcelona', 'Rome',
       'Bucharest', 'Budapest', 'Milan', 'Prague', 'Istanbul', 'Moscow',
       'Munich', 'Berlin', 'Paris', 'Brussels', 'Warsaw', 'Birmingham',
       'London', 'Sofia'], dtype='<U10')

# 4. Mating
For each pair of  parents we'll generate an offspring pair. Since we cannot repeat cities what we'll do is copy a random chunk from one progenitor and fill the blanks with the other progenitor.

In [8]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)
new_population_set[0]

array(['Warsaw', 'London', 'Brussels', 'Moscow', 'Munich', 'Kyiv', 'Rome',
       'Budapest', 'Vienna', 'Milan', 'Birmingham', 'Berlin', 'Minsk',
       'Barcelona', 'Istanbul', 'Bucharest', 'Paris', 'Amsterdam',
       'Prague', 'Sofia'], dtype='<U10')

# 5. Mutation
Now for each element of the new population we add a random chance of swapping

In [9]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
mutated_pop[0]

array(['Amsterdam', 'London', 'Brussels', 'Moscow', 'Warsaw', 'Kyiv',
       'Vienna', 'Budapest', 'Rome', 'Birmingham', 'Milan', 'Berlin',
       'Minsk', 'Barcelona', 'Istanbul', 'Munich', 'Paris', 'Bucharest',
       'Prague', 'Sofia'], dtype='<U10')

# 6. Stopping
To select the stopping criteria we'll need to create a loop to stop first. Then I'll set it to loop at 1000 iterations.

In [10]:
best_solution = [-1,np.inf,np.array([])]
for i in range(10000):
    if i%100==0: print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)
    
    mutated_pop = mutate_population(new_population_set)

0 590.7475243141516 783.9835797228405 31/01/21 17:48
100 645.0117218700279 788.9669110984397 31/01/21 17:48
200 624.2691069811408 784.636830396509 31/01/21 17:49
300 633.3129586073175 790.2006133890638 31/01/21 17:49
400 631.826460654929 796.9230192865534 31/01/21 17:49
500 645.9317975670073 787.2470883084593 31/01/21 17:49
600 657.2639223822969 785.1796360163803 31/01/21 17:49
700 624.3188964054668 760.6078170374126 31/01/21 17:49
800 670.4313328313078 786.5939834033436 31/01/21 17:49
900 628.2745550840066 787.720537796438 31/01/21 17:49
1000 648.3650564761382 796.6298811945546 31/01/21 17:49
1100 651.3471122404861 783.0413285273546 31/01/21 17:49
1200 587.0383496035005 798.0197123766501 31/01/21 17:49
1300 630.6548925000944 785.009995832812 31/01/21 17:49
1400 613.6284807662042 789.2173953870707 31/01/21 17:49
1500 670.2291246747689 789.8251134350099 31/01/21 17:49
1600 674.2928424413311 779.602603788733 31/01/21 17:49
1700 655.1298947849947 787.466527897643 31/01/21 17:49
1800 654.2

In [11]:
best_solution

[8714,
 487.5265019321421,
 array([['Birmingham', 'Rome', 'Berlin', 'Amsterdam', 'Brussels', 'Minsk',
         'Barcelona', 'London', 'Sofia', 'Budapest', 'Moscow', 'Paris',
         'Milan', 'Warsaw', 'Kyiv', 'Bucharest', 'Istanbul', 'Vienna',
         'Prague', 'Munich']], dtype='<U10')]